In [20]:
import pandas as pd

# Read in the data
folder_path = 'C:\\Users\\aoife\Documents\\Project\\filtered_data\\'

save_path = 'C:\\Users\\aoife\Documents\\Project\\'

# Read in normalisaed data

df = pd.read_csv(folder_path + 'minmax_normalised_data_without_surveys.csv')


print(df.head(5))

                          participantId  bodyMass_kg  height_m       bmi  \
0  00a55fb5-da33-4e2e-ae61-28f589fcc174     0.080718  0.578971  0.101313   
1  00fd4039-9b5e-4bbb-8295-4983a3f58371     0.094290  0.606985  0.107675   
2  02d5125e-684f-4166-a3b7-5df1bcfc1661     0.094290  0.625662  0.101342   
3  041d9018-9fcb-4cdc-9539-9021602ba36c     0.132150  0.653676  0.130121   
4  05fbf759-fcc0-4f5c-b871-f6a0d7f677b0     0.182151  0.691029  0.160489   

   mean_hr/s  hr_duration_(s)   max_hr   min_hr  totalSteps  steps_per_hour  \
0   0.359606         0.000544  0.44575  0.25425    0.018714        0.079119   
1   0.322549         0.000475  0.37925  0.28750    0.000256        0.004787   
2   0.276816         0.000741  0.44575  0.15425    0.045336        0.060711   
3   0.526849         0.003576  0.62075  0.19175    0.005714        0.074117   
4   0.321128         0.000475  0.37075  0.25825    0.031647        0.084616   

   totalDistance  dist_per_hour  ssq_score  
0       0.024815       

In [21]:
import numpy as np
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, make_scorer


In [22]:
# X - features

# X = df[['bodyMass_kg','height_m','bmi','mean_hr/s','hr_duration_(s)','max_hr','min_hr','totalSteps','steps_per_hour','totalDistance','dist_per_hour', 'alcohol_consumption', 'basic_expenses', 'caffeine', 'daily_activities', 'daily_smoking', 'education', 'flexible_work_hours', 'gender', 'good_life', 'hispanic','income', 'marital', 'race', 'smoking_status', 'menopause', 'recent_births', 'current_pregnant', 'work_schedule', 'alarm_dependency', 'driving_sleepy', 'falling_asleep', 'morning_person', 'nap_duration', 'sleep_lost', 'sleep_needed', 'sleep_partner', 'sleep_time_workday', 'sleep_time_weekend', 'wake_up_choices', 'wake_ups', 'weekly_naps', 'noise_light', 'stress_thinking', 'other_person', 'pain_discomfort', 'nightmares', 'bathroom_urges', 'other_reasons']].values

X = df[['bodyMass_kg','height_m','bmi','mean_hr/s','hr_duration_(s)','max_hr','min_hr','totalSteps','steps_per_hour','totalDistance','dist_per_hour']].values


# y - target
y = df['ssq_score'].values

groups = df['participantId'].values

In [23]:
# RMSE Scorer
def rmse_score(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse_score, greater_is_better=False)


In [24]:
gkf = GroupKFold(n_splits=10)


In [25]:
param_grid = {
    'activation': ['relu'],
    'batch_size': [100],
    'hidden_layer_sizes': [(50, 70)],
    'learning_rate': ['adaptive'],
    'learning_rate_init': [0.05],
    'max_iter': [500],
    'solver': ['sgd']
}

'''param_grid = {
    'hidden_layer_sizes': [(50,), (50,70), (100,)],
    'activation': ['relu'],
    'solver': ['sgd', 'adam'],
    'learning_rate_init': [0.0001, 0.01, 0.05],
    'batch_size': [50, 100],
    'learning_rate': ['invscaling', 'adaptive'],
    'max_iter': [500, 1000, 1500]
}'''

# 'tol': [1e-4, 1e-6]
# 'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1],

"param_grid = {\n    'hidden_layer_sizes': [(50,), (50,70), (100,)],\n    'activation': ['relu'],\n    'solver': ['sgd', 'adam'],\n    'learning_rate_init': [0.0001, 0.01, 0.05],\n    'batch_size': [50, 100],\n    'learning_rate': ['invscaling', 'adaptive'],\n    'max_iter': [500, 1000, 1500]\n}"

In [26]:
mlp_gs = GridSearchCV(MLPRegressor(max_iter=1000, random_state=42), param_grid, scoring=rmse_scorer, cv=gkf, verbose=2)

# Fit GridSearchCV
mlp_gs.fit(X, y, groups=groups) 


Fitting 10 folds for each of 1 candidates, totalling 10 fits


[CV] END activation=relu, batch_size=100, hidden_layer_sizes=(50, 70), learning_rate=adaptive, learning_rate_init=0.05, max_iter=500, solver=sgd; total time=   1.4s
[CV] END activation=relu, batch_size=100, hidden_layer_sizes=(50, 70), learning_rate=adaptive, learning_rate_init=0.05, max_iter=500, solver=sgd; total time=   1.5s
[CV] END activation=relu, batch_size=100, hidden_layer_sizes=(50, 70), learning_rate=adaptive, learning_rate_init=0.05, max_iter=500, solver=sgd; total time=   1.4s
[CV] END activation=relu, batch_size=100, hidden_layer_sizes=(50, 70), learning_rate=adaptive, learning_rate_init=0.05, max_iter=500, solver=sgd; total time=   2.7s
[CV] END activation=relu, batch_size=100, hidden_layer_sizes=(50, 70), learning_rate=adaptive, learning_rate_init=0.05, max_iter=500, solver=sgd; total time=   2.8s
[CV] END activation=relu, batch_size=100, hidden_layer_sizes=(50, 70), learning_rate=adaptive, learning_rate_init=0.05, max_iter=500, solver=sgd; total time=   2.1s
[CV] END a

GridSearchCV(cv=GroupKFold(n_splits=10),
             estimator=MLPRegressor(max_iter=1000, random_state=42),
             param_grid={'activation': ['relu'], 'batch_size': [100],
                         'hidden_layer_sizes': [(50, 70)],
                         'learning_rate': ['adaptive'],
                         'learning_rate_init': [0.05], 'max_iter': [500],
                         'solver': ['sgd']},
             scoring=make_scorer(rmse_score, greater_is_better=False),
             verbose=2)

In [27]:
# Best hyperparameters
print("Best Parameters:", mlp_gs.best_params_)

# Best RMSE score
print("Best RMSE Score:", -mlp_gs.best_score_)

# Best model
best_model = mlp_gs.best_estimator_


Best Parameters: {'activation': 'relu', 'batch_size': 100, 'hidden_layer_sizes': (50, 70), 'learning_rate': 'adaptive', 'learning_rate_init': 0.05, 'max_iter': 500, 'solver': 'sgd'}
Best RMSE Score: 0.3231646071633662
